In [ ]:
#
# Tests the ability to use springs layout selectively on subgraphs
# ... so, if you keep the placement of the nodes from an initial all-nodes layout
# ... then you can update the placement of a subset of the nodes
#
# However... because that's not the only transform done to make this all work
# (think about the treemap component placement), then there's always a scaling factor
# involved... and that breaks the ability to apply the spring layout to a subset of 
# nodes...
#

import polars   as pl
import networkx as nx
import random
import copy
import rtsvg
import linknode_graph_patterns
from rtsvg.polars_spring_layout import PolarsSpringLayout, WeakParabolicBottom
rt   = rtsvg.RACETrack()

g    = linknode_graph_patterns.LinkNodeGraphPatterns().__pattern_mesh__()
_lu_ = {'fm':[], 'to':[]}
for n in g.nodes():
    for nbor in g.neighbors(n): _lu_['fm'].append(n), _lu_['to'].append(nbor)
df        = pl.DataFrame(_lu_)
_to_fix_  = {'node_0_5', 'node_7_4', 'node_8_6', 'node_5_8'}

_relates_ = [('fm','to')]
_colors_  = {n:'red' for n in _to_fix_}
_psl_     = PolarsSpringLayout(g)
pos       = rt.treeMapGraphComponentPlacement(g, _psl_.results())
new_pos   = copy.deepcopy(pos)
for n in _to_fix_: new_pos[n] = (new_pos[n][0] + 100*(random.random()-0.5), new_pos[n][1] + 100*(random.random()-0.5))
new_new_pos = copy.deepcopy(new_pos)
new_new_pos = PolarsSpringLayout(g, pos=new_new_pos, static_nodes=set(g.nodes()) - _to_fix_).results()
params      = {'w':384, 'h':384, 'node_color':_colors_}
rt.tile([rt.link(df, _relates_, pos,         **params),
         rt.link(df, _relates_, new_pos,     **params),
         rt.link(df, _relates_, new_new_pos, **params)], spacer=10)

In [ ]:
_tiles_ = []
def makeTile():
    _lu_ = {'x':[], 'y':[], 'group':[]}
    x = 0.5
    while x < 20.0:
        _rms_ = _psl_.rootMeanSquareError(x, y_scale)
        _lu_['x'].append(x), _lu_['y'].append(_rms_), _lu_['group'].append('x')
        x += 0.5
    y = 0.5
    while y < 20.0:
        _rms_ = _psl_.rootMeanSquareError(x_scale, y)
        _lu_['x'].append(y), _lu_['y'].append(_rms_), _lu_['group'].append('y')
        y += 0.5
    return rt.xy(pl.DataFrame(_lu_), x_field='x', y_field='y', line_groupby_field='group', color_by='group', dot_size=None)

x_scale, y_scale = 1.0, 1.0 # initial guesses for the scaling factors
for i in range(10):
    def _xfn_(x): return _psl_.rootMeanSquareError(x, y_scale)
    _wpb_x_ = WeakParabolicBottom(_xfn_)
    x_scale = _wpb_x_.xys[-1][0]
    def _yfn_(y): return _psl_.rootMeanSquareError(x_scale, y)
    _wpb_y_ = WeakParabolicBottom(_yfn_)
    y_scale = _wpb_y_.xys[-1][0]
    print(f'({x_scale:9.3f} {y_scale:9.3f})')
    _tiles_.append(makeTile())
scaled_pos = {}
for x in new_pos:
    scaled_pos[x] = (new_pos[x][0] * x_scale, new_pos[x][1] * y_scale)
params = {'w':384, 'h':384, 'node_color':_colors_}
scaled_pos_with_springs = copy.deepcopy(scaled_pos)
scaled_pos_with_springs = PolarsSpringLayout(g, pos=scaled_pos_with_springs, static_nodes=set(g.nodes()) - _to_fix_).results()
rt.tile([rt.link(df, _relates_, scaled_pos,              **params),
         rt.link(df, _relates_, scaled_pos_with_springs, **params)], spacer=10)

In [ ]:
#rt.tile(_tiles_)

In [ ]:
from math import sqrt
class ConnectedGraphRMS(object):
    def __init__(self, g, pos):
        _lu_ = {'n':[], 'nbor':[], 'w':[], 'n_x':[], 'nbor_x':[], 'n_y':[], 'nbor_y':[]}
        for n in g.nodes():
            for nbor in g.neighbors(n):
                _w_                  =  1.0 if 'weight' not in g[n][nbor] else g[n][nbor]['weight']
                _xy_n_, _xy_nbor_    =  pos[n], pos[nbor]
                _lu_['n'].append(n),                 _lu_['nbor'].append(nbor),             _lu_['w'].append(_w_)
                _lu_['n_x'].append(_xy_n_[0]),       _lu_['n_y'].append(_xy_n_[1])
                _lu_['nbor_x'].append(_xy_nbor_[0]), _lu_['nbor_y'].append(_xy_nbor_[1])
        self.df = pl.DataFrame(_lu_)
    def rms(self, x_scale, y_scale):
        __n_x__, __n_y__       = pl.col('n_x'), pl.col('n_y')
        __nbor_x__, __nbor_y__ = pl.col('nbor_x'), pl.col('nbor_y')
        _df_ = self.df.with_columns(((__n_x__*x_scale - __nbor_x__*x_scale)**2 + (__n_y__*y_scale - __nbor_y__*y_scale)**2).alias('x2'))
        _df_ = _df_.with_columns((pl.col('x2')**0.5).alias('d'))
        _df_ = _df_.with_columns(((_df_['w'] - _df_['d'])**2).alias('rms'))
        return sqrt(_df_['rms'].sum()/len(_df_))

conn_rms = ConnectedGraphRMS(g, _psl_.results())

x_scale, y_scale = 1.0, 1.0 # initial guesses for the scaling factors
for i in range(10):
    def _xfn_(x): return conn_rms.rms(x, y_scale)
    _wpb_x_ = WeakParabolicBottom(_xfn_)
    x_scale = _wpb_x_.xys[-1][0]
    def _yfn_(y): return conn_rms.rms(x_scale, y)
    _wpb_y_ = WeakParabolicBottom(_yfn_)
    y_scale = _wpb_y_.xys[-1][0]
    print(f'({x_scale:9.3f} {y_scale:9.3f})')
    _tiles_.append(makeTile())
scaled_pos = {}
for x in new_pos:
    scaled_pos[x] = (new_pos[x][0] * x_scale, new_pos[x][1] * y_scale)
params = {'w':384, 'h':384, 'node_color':_colors_}
scaled_pos_with_springs = copy.deepcopy(scaled_pos)
scaled_pos_with_springs = PolarsSpringLayout(g, pos=scaled_pos_with_springs, static_nodes=set(g.nodes()) - _to_fix_).results()
rt.tile([rt.link(df, _relates_, scaled_pos,              **params),
         rt.link(df, _relates_, scaled_pos_with_springs, **params)], spacer=10)
